# 05 - Running Sagemaker Hyperparameter optimisation with your ML Script

You want to test various hyperparameters by testing and evaluating your module multiple times. SageMaker can do this by creating many jobs in parallel.

In [18]:
import os
import boto3
from sagemaker import Session
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.tuner import HyperparameterTuner
from sagemaker.tuner import CategoricalParameter, ContinuousParameter
from sagemaker.analytics import HyperparameterTuningJobAnalytics

## AWS Session

In [6]:
boto3_session = boto3.Session(region_name=os.environ.get("DEMO_AWS_REGION"), profile_name=os.environ.get("DEMO_AWS_PROFILE_NAME"))

sagemaker_session = Session(boto_session=boto3_session)

account = os.environ.get("DEMO_AWS_ACCOUNT")  # sandbox-admin account
role = f"arn:aws:iam::{account}:role/service-role/AmazonSageMaker-ExecutionRole-20171129T145583"

## Upload data to S3

In [7]:
# Upload training data from local machine to S3
local_data_location = "../data"

data_location = sagemaker_session.upload_data(
    path=local_data_location, key_prefix="sagemaker_demo_data"
)

## Run Script

In [32]:
# define hyper parameter ranges
metric_definitions = [
    {"Name": "test_roc_auc", "Regex": "test_roc_auc: (.+?);$"},
]

# define range of parameters to optimise for
params_to_optimise = {
    "penalty": CategoricalParameter(["l1", "l2"]),
    "C": ContinuousParameter(min_value=0.001, max_value=10.0, scaling_type="Logarithmic"),
}

In [33]:
# define how to run
sklearn = SKLearn(
    entry_point='../04_sagemaker_ml/dummy_ml_script_with_args_for_sagemaker.py',
    train_instance_type="ml.m5.large",
    role=role,
    sagemaker_session=sagemaker_session,
)

tuner = HyperparameterTuner(
    estimator=sklearn,
    max_jobs=10,
    max_parallel_jobs=3,
    hyperparameter_ranges=params_to_optimise,
    strategy="Bayesian",
    metric_definitions=metric_definitions,
    objective_metric_name="test_roc_auc",
    objective_type="Maximize",
    base_tuning_job_name="sklearn-demo-hyperopt",
)

In [34]:
# run script
tuner.fit({"train": data_location})

## Look at results

In [37]:
job_analytics = HyperparameterTuningJobAnalytics(
    sagemaker_session=sagemaker_session,
    hyperparameter_tuning_job_name="sklearn-demo-hyperop-200219-1517",
)
job_analytics_df = job_analytics.dataframe()

In [38]:
job_analytics_df

,C,penalty,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
0,0.001098,"""l1""",sklearn-demo-hyperop-200219-1517-010-83ffd497,Completed,0.995464,2020-02-19 15:28:04+00:00,2020-02-19 15:29:02+00:00,58.0
1,0.001299,"""l1""",sklearn-demo-hyperop-200219-1517-009-e1577be8,Completed,0.995464,2020-02-19 15:25:53+00:00,2020-02-19 15:26:58+00:00,65.0
2,6.899607,"""l2""",sklearn-demo-hyperop-200219-1517-008-4b7d0b6f,Completed,0.995322,2020-02-19 15:25:42+00:00,2020-02-19 15:26:46+00:00,64.0
3,0.005127,"""l2""",sklearn-demo-hyperop-200219-1517-007-37f3baf2,Completed,0.995399,2020-02-19 15:25:23+00:00,2020-02-19 15:26:20+00:00,57.0
4,0.003220,"""l2""",sklearn-demo-hyperop-200219-1517-006-2283b1d1,Completed,0.995401,2020-02-19 15:22:37+00:00,2020-02-19 15:23:41+00:00,64.0
5,0.002142,"""l1""",sklearn-demo-hyperop-200219-1517-005-d0b0eb7a,Completed,0.995464,2020-02-19 15:22:30+00:00,2020-02-19 15:23:27+00:00,57.0
6,0.002936,"""l2""",sklearn-demo-hyperop-200219-1517-004-353103aa,Completed,0.995401,2020-02-19 15:22:55+00:00,2020-02-19 15:23:57+00:00,62.0
7,0.008712,"""l1""",sklearn-demo-hyperop-200219-1517-003-7c2eec42,Completed,0.995464,2020-02-19 15:19:36+00:00,2020-02-19 15:20:38+00:00,62.0
8,0.001730,"""l1""",sklearn-demo-hyperop-200219-1517-002-9faf0213,Completed,0.995464,2020-02-19 15:19:15+00:00,2020-02-19 15:20:23+00:00,68.0
9,0.127057,"""l1""",sklearn-demo-hyperop-200219-1517-001-1bc75e4f,Completed,0.995435,2020-02-19 15:19:06+00:00,2020-02-19 15:20:04+00:00,58.0
